In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/MyDrive/Challenge1/


Mounted at /gdrive
/gdrive/MyDrive/Challenge1


In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
# Random seed for reproducibility
seed = 127
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch = 64

data_gen = ImageDataGenerator( 
                              horizontal_flip = True,
                              rotation_range = 10,
                              fill_mode='reflect',
                              channel_shift_range=10,
                              zoom_range = [1.0,1.1],
                              width_shift_range = 10,
                              height_shift_range = 5,
                              preprocessing_function = tf.keras.applications.efficientnet.preprocess_input

)

validation_gen = ImageDataGenerator(preprocessing_function = tf.keras.applications.efficientnet.preprocess_input

)


train_gen = data_gen.flow_from_directory(directory="split-data/train",
                                                           target_size=(96,96),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=batch,
                                                           shuffle=True,
                                                           seed=seed) # set as training data

val_gen = validation_gen.flow_from_directory(directory="split-data/val",
                                                           target_size=(96,96),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=batch,
                                                           shuffle=True,
                                                           seed=seed) # set as validation data

Found 3183 images belonging to 8 classes.
Found 351 images belonging to 8 classes.


In [ ]:
effnet = tf.keras.applications.efficientnet.EfficientNetB6(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3)
)
effnet.summary()
#tfk.utils.plot_model(supernet)

Model: "efficientnetb6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 rescaling_1 (Rescaling)        (None, 96, 96, 3)    0           ['input_5[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 96, 96, 3)   7           ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                                  
 tf.math.truediv_1 (TFOpLambda)  (None, 96, 96, 3)   0           ['normalization_1[0]

In [ ]:
# Use the supernet as feature extractor
effnet.trainable = True

inputs = tfk.Input(shape=(96,96,3))
# x = tfkl.Resizing(96, 96, interpolation="bicubic")(inputs)
x = effnet(inputs)
#x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.GlobalAveragePooling2D()(x)

x = tfkl.Dense(
    512, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)

x = tfkl.Dropout(0.3, seed=seed)(x)

x = tfkl.Dense(
    512, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)

x = tfkl.Dropout(0.3, seed=seed)(x)

outputs = tfkl.Dense(
    8, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')


for i, layer in enumerate(tl_model.get_layer('efficientnetb6').layers):
   print(i, layer.name, layer.trainable)

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')


0 input_5 True
1 rescaling_1 True
2 normalization_1 True
3 tf.math.truediv_1 True
4 stem_conv_pad True
5 stem_conv True
6 stem_bn True
7 stem_activation True
8 block1a_dwconv True
9 block1a_bn True
10 block1a_activation True
11 block1a_se_squeeze True
12 block1a_se_reshape True
13 block1a_se_reduce True
14 block1a_se_expand True
15 block1a_se_excite True
16 block1a_project_conv True
17 block1a_project_bn True
18 block1b_dwconv True
19 block1b_bn True
20 block1b_activation True
21 block1b_se_squeeze True
22 block1b_se_reshape True
23 block1b_se_reduce True
24 block1b_se_expand True
25 block1b_se_excite True
26 block1b_project_conv True
27 block1b_project_bn True
28 block1b_drop True
29 block1b_add True
30 block1c_dwconv True
31 block1c_bn True
32 block1c_activation True
33 block1c_se_squeeze True
34 block1c_se_reshape True
35 block1c_se_reduce True
36 block1c_se_expand True
37 block1c_se_excite True
38 block1c_project_conv True
39 block1c_project_bn True
40 block1c_drop True
41 block1c_

In [ ]:
#Unfreeze and fine tune some layers (keeping bn layers freezed)

# for layer in tl_model.get_layer('efficientnetb6').layers[457:]:
#     if not isinstance(layer, tfkl.BatchNormalization):
#         layer.trainable = True


# for i, layer in enumerate(ft_model.get_layer('efficientnetb6').layers):
#    print(i, layer.name, layer.trainable)

In [ ]:
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetb6 (Functional)  (None, 3, 3, 2304)       40960143  
                                                                 
 global_average_pooling2d_4   (None, 2304)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_12 (Dense)            (None, 512)               1180160   
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 512)               262656    
                                                             

In [ ]:
# Create folders and callbacks and fit
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)

epochs = 200
# Train the model
history = tl_model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = val_gen,
    callbacks = callbacks,
).history

Epoch 1/200
50/50 [==============================] - 65s 685ms/step - loss: 1.4599 - accuracy: 0.4857 - val_loss: 1.9863 - val_accuracy: 0.3248
Epoch 2/200
50/50 [==============================] - 28s 548ms/step - loss: 0.8865 - accuracy: 0.6949 - val_loss: 3.5083 - val_accuracy: 0.5043
Epoch 3/200
50/50 [==============================] - 28s 558ms/step - loss: 0.6753 - accuracy: 0.7725 - val_loss: 1.7708 - val_accuracy: 0.6467
Epoch 4/200
50/50 [==============================] - 28s 551ms/step - loss: 0.5155 - accuracy: 0.8234 - val_loss: 1.3518 - val_accuracy: 0.6952
Epoch 5/200
50/50 [==============================] - 28s 550ms/step - loss: 0.4798 - accuracy: 0.8473 - val_loss: 2.0695 - val_accuracy: 0.6724
Epoch 6/200
50/50 [==============================] - 28s 548ms/step - loss: 0.3692 - accuracy: 0.8847 - val_loss: 0.7434 - val_accuracy: 0.8234
Epoch 7/200
50/50 [==============================] - 28s 559ms/step - loss: 0.3199 - accuracy: 0.8988 - val_loss: 0.8054 - val_accuracy:

In [ ]:
# Evaluate on validation set
model_test_metrics = tl_model.evaluate(val_gen, return_dict=True)

print()
print("Test metrics without data augmentation")
print(model_test_metrics)

6/6 [==============================] - 1s 117ms/step - loss: 0.5697 - accuracy: 0.8860

Test metrics without data augmentation
{'loss': 0.5696983933448792, 'accuracy': 0.8860399127006531}


In [ ]:
tl_model.save("effnet6/effnet6_ftl")